In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd 
import codecs
import gc

In [2]:
os.chdir("C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\cic\\dataset\\Original_dataset-notuse")


In [3]:
def _to_utf8(filename: str, encoding="latin1", blocksize=1048576):
    tmpfilename = filename + ".tmp"
    with codecs.open(filename, "r", encoding) as source:
        with codecs.open(tmpfilename, "w", "utf-8") as target:
            while True:
                contents = source.read(blocksize)
                if not contents:
                    break
                target.write(contents)
    os.rename(filename, filename + '_old')
    # replace the original file
    os.rename(tmpfilename, filename)

In [6]:
os.remove('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv_old')
df1=pd.read_csv("Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")

file_name = "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
_to_utf8(file_name)
df6=pd.read_csv(file_name)

df7=pd.read_csv("Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("Wednesday-workingHours.pcap_ISCX.csv")


df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8
gc.collect()
nRow, nCol = df.shape
print(f'The table has {nRow} rows and {nCol} columns')

C:\Users\Kotani Lab\AppData\Local\Temp\ipykernel_5660\1739749492.py:10: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df6=pd.read_csv(file_name)


The table has 3119345 rows and 85 columns


In [7]:
def categorize_port(port):
    if 0 <= port <= 1023:
        return 1
    elif 1024 <= port <= 49151:
        return 2
    elif 49152 <= port <= 65535:
        return 3
    else:
        return None  # or however you want to handle out-of-range values

In [8]:
col_list = []
for i, name in enumerate(df.columns):
    if name[0] == ' ':
        name = name[1:]
    if ' ' in name:
        name = name.replace(' ', '_')
    col_list.append(name)
df.columns = col_list

In [9]:
df['Destination_Port'] = df['Destination_Port'].apply(categorize_port)

In [10]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
cols_with_nan = df.columns[df.isna().any()].tolist()
print("Columns containing NaN values:", cols_with_nan)


Columns containing NaN values: []


In [11]:
list = df.columns.tolist()
list = [x.lower() for x in list]
df.columns = list
df = df.rename(columns = {'destination_port': 'destination_port_priority'})

In [12]:

# One-hot encode the 'destination_port_priority' column
temp = pd.get_dummies(df['destination_port_priority'], prefix='destination_port_priority', dtype=int)

# Drop the original 'destination_port_priority' column and append the dummies
df = df.drop('destination_port_priority', axis=1).join(temp)

In [13]:
del temp

In [14]:
def _renaming_class_label(df: pd.DataFrame):
    labels = {"Web Attack \x96 Brute Force": "Web Attack-Brute Force",
              "Web Attack \x96 XSS": "Web Attack-XSS",
              "Web Attack \x96 Sql Injection": "Web Attack-Sql Injection"}

    for old_label, new_label in labels.items():
        df.label.replace(old_label, new_label, inplace=True)

# Renaming labels
_renaming_class_label(df)

In [24]:
df['timestamp']

0          7/7/2017 3:30
0          7/7/2017 3:30
0          7/7/2017 3:30
0          7/7/2017 3:30
0          7/7/2017 3:30
               ...      
692698    5/7/2017 12:10
692699     5/7/2017 3:02
692700    5/7/2017 10:06
692701     5/7/2017 1:19
692702     5/7/2017 2:43
Name: timestamp, Length: 16601424, dtype: object

In [15]:
gc.collect()

df = df.sort_values(by='timestamp')
non_numerical_columns = df.select_dtypes(exclude=['number']).columns


In [16]:
df = df.drop(['source_port'], axis = 1)

In [17]:
df = df.drop(non_numerical_columns.drop(['label', 'timestamp']), axis = 1)


In [18]:
gc.collect()

0

In [22]:
df.to_csv('CIC_IDS2017.csv', index=False)

,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,...,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,destination_port_priority_1.0,destination_port_priority_2.0,destination_port_priority_3.0
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
355323,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0,0,1
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0,1,0
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
355323,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
355323,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,1,0,0
193126,6.0,03/07/2017 01:00:02,3.0,2.0,0.0,37.0,0.0,37.0,0.0,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0,1,0
